In [1]:
pip install pyspark

     |████████████████████████████████| 204.7MB 59kB/s 
     |████████████████████████████████| 204kB 42.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=67d7ec1443275a958585515da667ccff3bee719925834d34a2e3bcb6bf0e8c1e
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark


In [2]:
from pyspark import SparkContext

Create array of names and publish name and length of each name.

In [3]:
tmp2 = ['Mary', 'had', 'a', 'little', 'lamb', 'whose', 'fleece', 'was', 'white']
sc = SparkContext.getOrCreate()
array = sc.parallelize(tmp2)
array.map(lambda x: (x, len(x))).collect()

[('Mary', 4),
 ('had', 3),
 ('a', 1),
 ('little', 6),
 ('lamb', 4),
 ('whose', 5),
 ('fleece', 6),
 ('was', 3),
 ('white', 5)]

Create array of names and find max, min, standard deviation, variance of length of each element in array.

In [4]:
tmp2 = ['Mary', 'had', 'a', 'little', 'lamb', 'whose', 'fleece', 'was', 'white']
sc = SparkContext.getOrCreate()
array = sc.parallelize(tmp2).map(lambda x: len(x))
array.stats()

(count: 9, mean: 4.111111111111111, stdev: 1.5234788000891208, max: 6.0, min: 1.0)

Write a program to pull emp table data(name, age ,salary) from SQL server instance and find
name, age of employees whose salary is greater than avg salary of employee having age &gt;60
and then store it into spark dataframe

In [ ]:
import pyodbc

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-AQC7I1Q;'
                     'Database=Test;'
                     'Trusted_Connection=yes;')



In [ ]:
import pandas as pd
query='select name,age from [Test].[dbo].[emp] where salary > (select AVG(salary) from emp where age >60) and age>60;'
pdf = pd.read_sql(query,conn)
sparkDF = spark.createDataFrame(pdf)
sparkDF.show()

+-----+---+
| name|age|
+-----+---+
|Radhe| 64|
+-----+---+



Use data set “ocp_data.tsv” attached in sheet (data is delimited with tab space)
1. List no. of shows per year.
2. List no. of shows per year for every occupation
3. Find top occupation for every year.
4. Which guest have attended maximum shows
5. List occupation under each group
6. How many show per group of year and occupation

In [5]:
# Create connection
import pyspark
from pyspark.sql  import SparkSession
spark = SparkSession.builder.appName('OCP').getOrCreate()

In [6]:
dataframe = spark.read.csv('/OCP_data.tsv', sep='\t', inferSchema=True, header=True)

In [7]:
dataframe.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- GoogleKnowlege_Occupation: string (nullable = true)
 |-- Show: string (nullable = true)
 |-- Group: string (nullable = true)
 |-- Raw_Guest_List: string (nullable = true)



In [8]:
dataframe.show()

+----+-------------------------+----------+--------+--------------------+
|YEAR|GoogleKnowlege_Occupation|      Show|   Group|      Raw_Guest_List|
+----+-------------------------+----------+--------+--------------------+
|1999|                    actor|01-11-1999|  Acting|      Michael J. Fox|
|1999|                 Comedian|01-12-1999|  Comedy|     Sandra Bernhard|
|1999|       television actress|   1/13/99|  Acting|       Tracey Ullman|
|1999|             film actress|   1/14/99|  Acting|    Gillian Anderson|
|1999|                    actor|   1/18/99|  Acting|    David Alan Grier|
|1999|                    actor|   1/19/99|  Acting|     William Baldwin|
|1999|          Singer-lyricist|   1/20/99|Musician|       Michael Stipe|
|1999|                    model|   1/21/99|   Media|      Carmen Electra|
|1999|                    actor|   1/25/99|  Acting|     Matthew Lillard|
|1999|        stand-up comedian|   1/26/99|  Comedy|         David Cross|
|1999|                  actress|   1/2

In [9]:
# List no. of shows per year.
dataframe.groupBy('Year').count().show()

+----+-----+
|Year|count|
+----+-----+
|2003|  166|
|2007|  141|
|2015|  100|
|2006|  161|
|2013|  166|
|2014|  163|
|2004|  164|
|2012|  164|
|2009|  163|
|2001|  157|
|2005|  162|
|2000|  169|
|2010|  165|
|2011|  163|
|2008|  164|
|1999|  166|
|2002|  159|
+----+-----+



In [10]:
# List no. of shows per year for every occupation
dataframe.groupBy('Year', 'GoogleKnowlege_Occupation').count().show()

+----+-------------------------+-----+
|Year|GoogleKnowlege_Occupation|count|
+----+-------------------------+-----+
|2001|             Screenwriter|    1|
|2004|                scientist|    1|
|2004|     television presenter|    1|
|2005|     American football...|    1|
|2006|     Former member of ...|    1|
|2007|                Columnist|    2|
|2011|     United States Sen...|    1|
|2000|           game show host|    1|
|2004|                   author|    6|
|2008|                 reporter|    1|
|2009|          television host|    4|
|2010|        singer-songwriter|    1|
|2011|         political figure|    2|
|2001|     television person...|    1|
|2001|        stand-up comedian|    1|
|2002|                   Author|    1|
|2004|     civil rights acti...|    1|
|2007|               journalist|   10|
|2009|     United States Amb...|    1|
|2011|       political satirist|    1|
+----+-------------------------+-----+
only showing top 20 rows



In [11]:
# Find top occupation for every year.
df = dataframe.groupBy('Year', 'GoogleKnowlege_Occupation').count()
df = df.sort(df['count'].desc())
df.show()

+----+-------------------------+-----+
|Year|GoogleKnowlege_Occupation|count|
+----+-------------------------+-----+
|2002|                    actor|   64|
|2001|                    actor|   62|
|2000|                    actor|   61|
|1999|                    actor|   53|
|2003|                    actor|   47|
|1999|                  actress|   42|
|2013|                    actor|   37|
|2006|                    actor|   33|
|2014|                    actor|   31|
|2004|                    actor|   31|
|2010|                    actor|   30|
|2000|                  actress|   29|
|2011|                    actor|   29|
|2005|                    actor|   25|
|2003|                  actress|   24|
|2013|                  actress|   21|
|2002|                  actress|   20|
|2012|                    actor|   20|
|2008|                    actor|   20|
|2008|               journalist|   19|
+----+-------------------------+-----+
only showing top 20 rows



In [12]:
df = df.dropDuplicates(['Year'])
df.sort(df['count'].desc()).show()

+----+-------------------------+-----+
|Year|GoogleKnowlege_Occupation|count|
+----+-------------------------+-----+
|2002|                    actor|   64|
|2001|                    actor|   62|
|2000|                    actor|   61|
|1999|                    actor|   53|
|2003|                    actor|   47|
|2013|                    actor|   37|
|2006|                    actor|   33|
|2014|                    actor|   31|
|2004|                    actor|   31|
|2010|                    actor|   30|
|2011|                    actor|   29|
|2005|                    actor|   25|
|2012|                    actor|   20|
|2008|                    actor|   20|
|2009|                    actor|   19|
|2015|                    actor|   19|
|2007|                    actor|   15|
+----+-------------------------+-----+



In [13]:
# Which guest have attended maximum shows
df1 = dataframe.groupBy('Raw_Guest_List').count()
df1.sort(df1['count'].desc()).show()

+-----------------+-----+
|   Raw_Guest_List|count|
+-----------------+-----+
|   Fareed Zakaria|   19|
|      Denis Leary|   17|
|   Brian Williams|   16|
|    Ricky Gervais|   13|
|        Paul Rudd|   13|
|       Tom Brokaw|   12|
|    Bill O'Reilly|   10|
|    Richard Lewis|   10|
|     Will Ferrell|   10|
|       Reza Aslan|   10|
|     Sarah Vowell|    9|
|Maggie Gyllenhaal|    8|
|     Adam Sandler|    8|
|    Mike Huckabee|    8|
|      Ben Affleck|    8|
|       Louis C.K.|    8|
|     Bill Clinton|    7|
|     Steve Carell|    7|
|      Jeff Garlin|    7|
|      Colin Firth|    7|
+-----------------+-----+
only showing top 20 rows



In [14]:
# List occupation under each group
from pyspark.sql import functions as F
dataframe.groupBy('Group').agg(F.collect_list('GoogleKnowlege_Occupation')).show()

+--------------+---------------------------------------+
|         Group|collect_list(GoogleKnowlege_Occupation)|
+--------------+---------------------------------------+
|       Science|                   [neurosurgeon, sc...|
|      Musician|                   [Singer-lyricist,...|
|            NA|                   [NA, NA, NA, NA, ...|
|Political Aide|                   [former press sec...|
|    Government|                   [Diplomat, Former...|
|      Military|                   [military officer...|
|    Consultant|                   [political psycho...|
|     Athletics|                   [football player,...|
|        Clergy|                   [minister, pastor...|
|      Academic|                   [professor, Profe...|
|         Media|                   [model, televisio...|
|         media|                   [news anchor, new...|
|          Misc|                   [chef, Lawyer, la...|
|      Business|                   [businesswoman, B...|
|        Acting|               

In [15]:
# How many show per group of year and occupation
dataframe.groupBy('Year', 'GoogleKnowlege_Occupation').count().show()

+----+-------------------------+-----+
|Year|GoogleKnowlege_Occupation|count|
+----+-------------------------+-----+
|2001|             Screenwriter|    1|
|2004|                scientist|    1|
|2004|     television presenter|    1|
|2005|     American football...|    1|
|2006|     Former member of ...|    1|
|2007|                Columnist|    2|
|2011|     United States Sen...|    1|
|2000|           game show host|    1|
|2004|                   author|    6|
|2008|                 reporter|    1|
|2009|          television host|    4|
|2010|        singer-songwriter|    1|
|2011|         political figure|    2|
|2001|     television person...|    1|
|2001|        stand-up comedian|    1|
|2002|                   Author|    1|
|2004|     civil rights acti...|    1|
|2007|               journalist|   10|
|2009|     United States Amb...|    1|
|2011|       political satirist|    1|
+----+-------------------------+-----+
only showing top 20 rows

